In [51]:
"""
Este fichero obtendrá el dataSet y
modelará los datos para su posterior uso en entrenamiento y validacion
"""




import findspark
findspark.init()

import sys, os, re
import json
import codecs


import pyspark
import sys
import decimal
import time
import datetime
import calendar
import json
import re
import base64
from array import array

if sys.version >= "3":
    long = int
    basestring = unicode = str

from py4j.protocol import register_input_converter
from py4j.java_gateway import JavaClass

from pyspark.serializers import CloudPickleSerializer
from pyspark.ml.regression import LinearRegression

import pyspark.sql.types
#from pyspark.sql import Row
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import matplotlib.pyplot as plt
import six
from types import *
import pandas as pd
import numpy as np
import pandas as pd
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import VectorAssembler, SQLTransformer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.sql.functions import col,sum


from pyspark.sql.functions import *
from pyspark.sql.types import *

from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession

#Se crea una sescion de spark
#Importante tras ejecucion hacer sc.stop()

import findspark
findspark.init()
sc = SparkContext('local')
spark = SparkSession(sc)


#El fichero .csv no tiene cabeceras
#Se define el esquema que va a tener el data frame
schema = StructType([
    StructField( 'srcip', StringType(), True),
    StructField('sport', IntegerType(), True),
    StructField('dstip', StringType(), True),
    StructField('dsport',IntegerType(), True),
    StructField('proto',StringType() , True),
    StructField('state', StringType(), True),
    StructField('dur', IntegerType(), True),
    StructField('sbytes', IntegerType(), True),
    StructField('dbytes', IntegerType(), True),
    StructField('sttl', IntegerType(), True),
    StructField('dttl', IntegerType(), True),
    StructField('sloss', IntegerType(), True),
    StructField('dloss', IntegerType(), True),
    StructField('service', StringType(), True),
    StructField('Sload', IntegerType(), True),
    StructField('Dload', IntegerType(), True),
    StructField('Spkts',IntegerType(), True),
    StructField('Dpkts',IntegerType(), True),
    StructField('swin', IntegerType(), True),
    StructField('dwin', IntegerType(), True),
    StructField('stcpb', IntegerType(), True),
    StructField('dtcpb', IntegerType(), True),
    StructField('smeansz', IntegerType(), True),
    StructField('dmeansz', IntegerType(), True),
    StructField('trans_depth',IntegerType(), True),
    StructField('res_bdy_len',IntegerType(), True),
    StructField('Sjit',IntegerType(), True),
    StructField('Djit',IntegerType(), True),
    StructField('Stime',IntegerType(), True),
    StructField('Ltime', IntegerType(), True),
    StructField('Sintpkt',IntegerType(), True),
    StructField('Dintpkt',IntegerType(), True),
    StructField('tcprtt',IntegerType(), True),
    StructField('synack',IntegerType(), True),
    StructField('ackdat',IntegerType(), True),
    StructField('is_sm_ips_ports',IntegerType(), True),
    StructField('ct_state_ttl',IntegerType(), True),
    StructField('ct_flw_http_mthd',IntegerType(), True),
    StructField('is_ftp_login',IntegerType(), True),
    StructField('ct_ftp_cm',IntegerType(), True),
    StructField('ct_srv_src',IntegerType(), True),
    StructField('ct_srv_dst',IntegerType(), True),
    StructField('ct_dst_ltm',IntegerType(), True),
    StructField('ct_src_ ltm',IntegerType(), True),
    StructField('ct_src_dport_ltm',IntegerType(), True),
    StructField('ct_dst_sport_ltm',IntegerType(), True),
    StructField('ct_dst_src_ltm',IntegerType(), True),
    StructField('attack_cat',StringType(), True),
    StructField('label',StringType(), True)])




"""
Datos estarán posiblemente en formato CSV
"""

# Leer un CSV a un DataFrame, PONER EL BUENO

dataSet = spark.read.format("csv").option("header", "true").schema(schema).option("mode", "DROPMALFORMED").load("UNSW.csv")


# Conveniente una visualizacion para ver como están los datos.
# Por ejemplo convertir a JSON y visualizar

# Contar número de tramas o de comunicaciones en el dataSet


print("features iniciales-------------------------------")
print(len(dataSet.columns))
print((dataSet.show(6)))

#Procedemos a la vectorizacion de las caracteristicas




dataSet = dataSet.sample(False, 0.55, seed=0)

dataSet.count()

dataSet = dataSet.na.replace([" Fuzzers "," Shellcode ","Backdoor"," Reconnaissance "],
                             [" Fuzzers","Shellcode","Backdoors","Reconnaissance"],"attack_cat")
dataSet.select("attack_cat").distinct().show()


features iniciales-------------------------------
49
+------------+-----+--------------+------+-----+-----+---+------+------+----+----+-----+-----+-------+-----+-----+-----+-----+----+----+-----+-----+-------+-------+-----------+-----------+----+----+----------+----------+-------+-------+------+------+------+---------------+------------+----------------+------------+---------+----------+----------+----------+-----------+----------------+----------------+--------------+----------+-----+
|       srcip|sport|         dstip|dsport|proto|state|dur|sbytes|dbytes|sttl|dttl|sloss|dloss|service|Sload|Dload|Spkts|Dpkts|swin|dwin|stcpb|dtcpb|smeansz|dmeansz|trans_depth|res_bdy_len|Sjit|Djit|     Stime|     Ltime|Sintpkt|Dintpkt|tcprtt|synack|ackdat|is_sm_ips_ports|ct_state_ttl|ct_flw_http_mthd|is_ftp_login|ct_ftp_cm|ct_srv_src|ct_srv_dst|ct_dst_ltm|ct_src_ ltm|ct_src_dport_ltm|ct_dst_sport_ltm|ct_dst_src_ltm|attack_cat|label|
+------------+-----+--------------+------+-----+-----+---+------+------

In [52]:
dataSet.count()

1397008

In [53]:
training_data, test_data = dataSet.randomSplit([0.75, 0.25])

In [55]:
training_data.count()

174

In [54]:
test_data.count()

50

In [57]:
splits = dataSet.randomSplit([0.7, 0.3])
train = splits[0]
test = splits[1]

In [58]:
test.count()

74

In [59]:
train.count()

150

In [8]:
#Metricas antiguas
from pyspark.ml.feature import StringIndexer, VectorAssembler, OneHotEncoderEstimator, StringIndexerModel
from pyspark.ml.classification import RandomForestClassificationModel
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import MultilayerPerceptronClassificationModel
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
base_path = "."

string_indexer_models = {}
for column in ["srcip", "sport","dstip","dsport","proto", "state","dur","sbytes","dbytes","sttl","dttl","sloss","dloss","service","Sload","Dload","Spkts","Dpkts","swin","dwin","stcpb","dtcpb","smeansz","dmeansz","trans_depth","res_bdy_len","Sjit","Djit", "Stime","Ltime","Sintpkt","Dintpkt","tcprtt","synack","ackdat","is_sm_ips_ports","ct_state_ttl","ct_flw_http_mthd","is_ftp_login","ct_ftp_cm","ct_srv_src","ct_srv_dst","ct_dst_ltm","ct_src_ ltm", "ct_src_dport_ltm","ct_dst_sport_ltm", "ct_dst_src_ltm", "label", "attack_cat"]:
        string_indexer_model_path = "{}/data/string_indexer_encoder/string_indexer_model_randomForest_{}.bin".format(
                  base_path,
                  column
                )
        string_indexer = StringIndexerModel.load(string_indexer_model_path)
        string_indexer_models[column] = string_indexer


for column in ["srcip", "sport","dstip","dsport","proto", "state","dur","sbytes","dbytes","sttl","dttl","sloss","dloss","service","Sload","Dload","Spkts","Dpkts","swin","dwin","stcpb","dtcpb","smeansz","dmeansz","trans_depth","res_bdy_len","Sjit","Djit", "Stime","Ltime","Sintpkt","Dintpkt","tcprtt","synack","ackdat","is_sm_ips_ports","ct_state_ttl","ct_flw_http_mthd","is_ftp_login","ct_ftp_cm","ct_srv_src","ct_srv_dst","ct_dst_ltm","ct_src_ ltm", "ct_src_dport_ltm","ct_dst_sport_ltm", "ct_dst_src_ltm", "label", "attack_cat"]:
    string_indexer_model = string_indexer_models[column]
    test_data = string_indexer_model.transform(test_data)

In [9]:
vector_assembler_path = "{}/data/numeric_vector_assembler_randomForest.bin".format(base_path)
vector_assembler = VectorAssembler.load(vector_assembler_path)

test_data = vector_assembler.transform(test_data)

In [10]:
model_path = "{}/data/RandomForest15.bin".format(
      base_path
  )

#model = MultilayerPerceptronClassificationModel.load(model_path)
model = RandomForestClassificationModel.load(model_path)
predictions = model.transform(test_data)

In [11]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Evaluate model using test data
predictions = model.transform(test_data)

from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(
  labelCol="attack_cat_index", metricName="accuracy"
)
accuracy = evaluator.evaluate(predictions)
print("Accuracy = {}".format(accuracy))

evaluator = MulticlassClassificationEvaluator(
  labelCol="attack_cat_index", metricName="weightedPrecision"
)
weightedPrecision = evaluator.evaluate(predictions)
print("weightedPrecision = {}".format(weightedPrecision))

evaluator = MulticlassClassificationEvaluator(
  labelCol="attack_cat_index", metricName="f1"
)
f1 = evaluator.evaluate(predictions)
print("f1 = {}".format(f1))

Accuracy = 0.9725986797790651
weightedPrecision = 0.9720251781229018
f1 = 0.9673113675680296


In [17]:
from pyspark.mllib.classification import LogisticRegressionWithLBFGS
from pyspark.mllib.util import MLUtils
from pyspark.mllib.evaluation import MulticlassMetrics

# Compute raw scores on the test set
predictionAndLabels = test_data.rdd.map(lambda lp: (float(model.predict(lp.features)), lp.label))

# Instantiate metrics object
metrics = MulticlassMetrics(predictionAndLabels)
# Overall statistics
precision = metrics.precision()
recall = metrics.recall()
f1Score = metrics.fMeasure()
print("Summary Stats")
print("Precision = %s" % precision)
print("Recall = %s" % recall)
print("F1 Score = %s" % f1Score)

# Statistics by class
labels = data.map(lambda lp: lp.label).distinct().collect()
for label in sorted(labels):
    print("Class %s precision = %s" % (label, metrics.precision(label)))
    print("Class %s recall = %s" % (label, metrics.recall(label)))
    print("Class %s F1 Measure = %s" % (label, metrics.fMeasure(label, beta=1.0)))

# Weighted stats
print("Weighted recall = %s" % metrics.weightedRecall)
print("Weighted precision = %s" % metrics.weightedPrecision)
print("Weighted F(1) Score = %s" % metrics.weightedFMeasure())
print("Weighted F(0.5) Score = %s" % metrics.weightedFMeasure(beta=0.5))
print("Weighted false positive rate = %s" % metrics.weightedFalsePositiveRate)


Traceback (most recent call last):
  File "/home/vagrant/spark/python/pyspark/serializers.py", line 590, in dumps
    return cloudpickle.dumps(obj, 2)
  File "/home/vagrant/spark/python/pyspark/cloudpickle.py", line 863, in dumps
    cp.dump(obj)
  File "/home/vagrant/spark/python/pyspark/cloudpickle.py", line 260, in dump
    return Pickler.dump(self, obj)
  File "/home/vagrant/anaconda/lib/python3.6/pickle.py", line 409, in dump
    self.save(obj)
  File "/home/vagrant/anaconda/lib/python3.6/pickle.py", line 476, in save
    f(self, obj) # Call unbound method with explicit self
  File "/home/vagrant/anaconda/lib/python3.6/pickle.py", line 751, in save_tuple
    save(element)
  File "/home/vagrant/anaconda/lib/python3.6/pickle.py", line 476, in save
    f(self, obj) # Call unbound method with explicit self
  File "/home/vagrant/spark/python/pyspark/cloudpickle.py", line 406, in save_function
    self.save_function_tuple(obj)
  File "/home/vagrant/spark/python/pyspark/cloudpickle.py", 

PicklingError: Could not serialize object: Py4JError: An error occurred while calling o1637.__getstate__. Trace:
py4j.Py4JException: Method __getstate__([]) does not exist
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:318)
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:326)
	at py4j.Gateway.invoke(Gateway.java:274)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:238)
	at java.lang.Thread.run(Thread.java:748)



In [11]:
#MÉTRICAS PARA LA NUEVA INGENIERÍA DE CARACTERÍSTICAS

from pyspark.ml.feature import StringIndexer, VectorAssembler, OneHotEncoderEstimator, StringIndexerModel
from pyspark.ml.classification import RandomForestClassificationModel
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import MultilayerPerceptronClassificationModel
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
base_path = "."

string_indexer_models = {}
for column in ["srcip","dstip","proto", "state","service","label", "attack_cat"]:
        string_indexer_model_path = "{}/data/string_indexer_RFT/string_indexer_model_RFT_{}.bin".format(
                  base_path,
                  column
                )
        string_indexer = StringIndexerModel.load(string_indexer_model_path)
        string_indexer_models[column] = string_indexer


for column in ["srcip","dstip","proto", "state","service","label", "attack_cat"]:
    string_indexer_model = string_indexer_models[column]
    test_data = string_indexer_model.transform(test_data)
    
    
    

In [12]:
vector_assembler_path = "{}/data/numeric_vector_assembler_RFT.bin".format(base_path)
vector_assembler = VectorAssembler.load(vector_assembler_path)

test_data = vector_assembler.transform(test_data)

In [13]:
model_path = "{}/data/RandomForestTree.bin".format(
      base_path
  )

#model = MultilayerPerceptronClassificationModel.load(model_path)
model = RandomForestClassificationModel.load(model_path)
predictions = model.transform(test_data)



In [17]:
predictionsAndLabels = predictions.select("prediction", "attack_cat_index_")

predictionAndLabels = predictionsAndLabels.rdd

In [18]:
print(predictionsAndLabels)

DataFrame[prediction: double, attack_cat_index_: double]


In [21]:
from pyspark.mllib.evaluation import MulticlassMetrics

metrics = MulticlassMetrics(predictionAndLabels)

In [22]:
metrics.confusionMatrix().toArray()

array([[ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 0.,  0., 44.]])

In [45]:
matrizDeConfusion = metrics.confusionMatrix()
print(matrizDeConfusion)

DenseMatrix([[ 1.,  0.,  0.],
             [ 1.,  0.,  0.],
             [ 0.,  0., 44.]])


In [28]:
"""
falsePositiveRate = metrics.falsePositiveRate(0)
print("falsePositiveRate")
print(falsePositiveRate)

precision = metrics.precision(1.0)
print("precision")
print(precision)

recall = metrics.recall(2.0)
print("recall")
print(recall)
#metrics.fMeasure(0.0, 2.0)
"""
accuracy = metrics.accuracy
print("accuracy")
print(accuracy)

weightedFalsePositiveRate = metrics.weightedFalsePositiveRate
print("weightedFalsePositiveRate")
print(weightedFalsePositiveRate)

weightedPrecision = metrics.weightedPrecision
print("weightedPrecision")
print(weightedPrecision)

weightedRecall = metrics.weightedRecall
print("weightedRecall")
print(weightedRecall)

#metrics.weightedFMeasure()

#metrics.weightedFMeasure(2.0)


accuracy
0.9782608695652174
weightedFalsePositiveRate
0.00048309178743961357
weightedPrecision
0.9673913043478262
weightedRecall
0.9782608695652174


In [44]:
falsePositiveRate = metrics.falsePositiveRate(3.0)
print("falsePositiveRate")
print(falsePositiveRate)


falsePositiveRate
0.0


In [46]:
a = predictions.select("prediction").distinct().show()
a= predictions.select("attack_cat_index_").distinct().show()

+----------+
|prediction|
+----------+
|       1.0|
|       9.0|
+----------+

+-----------------+
|attack_cat_index_|
+-----------------+
|              1.0|
|              3.0|
|              9.0|
+-----------------+



In [47]:
predictionsAndLabels.count()

46

In [48]:
predictions.count()

46

In [60]:
sc.stop()